![molprop](https://storage.googleapis.com/polaris-public/icons/icons8-bear-100-Molprop.png)
## Molecular representation benchmarks - MolProp250K

## Background

Molecular representations are crucial for understanding molecular structure, predicting properties, QSAR studies, toxicology and chemical modeling and other aspects in drug discovery tasks. Therefore, benchmarks for molecular representations are critical tools that drive progress in the field of computational chemistry and drug design. In recent years, many large models have been trained for learning molecular representation. The aim is to evaluate if large pretrained models are capable of predicting various “easy-to-compute” molecular properties. 

## Description of molecular properties
 The computed properties are molecular weight, fraction of sp3 carbon atoms (fsp3), number of rotatable bonds, topological polar surface area, computed logP, formal charge, number of charged atoms, refractivity and number of aromatic rings. These properties are widely used in molecule design and molecule prioritization.

## Data resource
**Reference**: https://pubs.acs.org/doi/10.1021/acs.jcim.5b00559 

**Raw data**: https://raw.githubusercontent.com/aspuru-guzik-group/chemical_vae/master/models/zinc_properties/250k_randm_zinc_drugs_clean_3.csv

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation

from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[2]
os.chdir(root)
sys.path.insert(0, str(root))
from utils.docs_utils import load_readme

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/lu.zhu/miniconda3/envs/po_dataset/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
org = "polaris"
data_name = "molprop"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"


owner = HubOwner(slug=org, type="organization")
owner

HubOwner(slug='polaris', external_id=None, type='organization')

In [4]:
BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_DIR = f"{gcp_root}/datasets"
FIGURE_DIR = f"{gcp_root}/figures"

## Load existing data

In [5]:
PATH = f"{gcp_root}/data/curation/{data_name}_curated.csv.gz"
table = pd.read_csv(PATH, compression="gzip")
table.columns

Index(['smiles', 'mw', 'fsp3', 'n_rotatable_bonds', 'tpsa', 'clogp',
       'formal_charge', 'n_charged_atoms', 'refractivity', 'n_aromatic_rings',
       'MOL_smiles', 'MOL_molhash_id', 'MOL_molhash_id_no_stereo',
       'MOL_num_stereoisomers', 'MOL_num_undefined_stereoisomers',
       'MOL_num_defined_stereo_center', 'MOL_num_undefined_stereo_center',
       'MOL_num_stereo_center', 'MOL_undefined_E_D', 'MOL_undefined_E/Z'],
      dtype='object')

## Below we specify the meta information of data columns

In [6]:
data_cols = [
    "mw",
    "fsp3",
    "n_rotatable_bonds",
    "tpsa",
    "clogp",
    "formal_charge",
    "n_charged_atoms",
    "refractivity",
    "n_aromatic_rings",
]

In [7]:
annotations = {
    "MOL_molhash_id": ColumnAnnotation(
        description="Molecular hash ID. See <datamol.mol.hash_mol>",
        modality="molecule",
    ),
    "MOL_smiles": ColumnAnnotation(
        description="Molecule Smiles string",
        modality="molecule",
    ),
    "mw": ColumnAnnotation(
        description="Molecular weight computed with <datamol.descriptor.mw>"
    ),
    "fsp3": ColumnAnnotation(
        description="Fraction of saturated carbons computed with <datamol.descriptor.fsp3>"
    ),
    "n_rotatable_bonds": ColumnAnnotation(
        description="A rotatable bond is defined as any single non-ring bond, attached to a non-terminal, non-hydrogen atom, computed with <datamol.descriptor.n_rotatable_bonds>"
    ),
    "tpsa": ColumnAnnotation(
        description="Topological polar surface area of a molecule is defined as the surface sum over all polar atoms or molecules, primarily oxygen and nitrogen, also including their attached hydrogen atoms. Computed with <datamol.descriptor.tpsa>"
    ),
    "clogp": ColumnAnnotation(
        description="Wildman-Crippen LogP value, computed with <datamol.descriptor.clogp>"
    ),
    "formal_charge": ColumnAnnotation(
        description="Formal Charge is a charge assigned to an atom under the assumption that all electrons in bonds are shared equally, computed with <datamol.descriptor.formal_charge>"
    ),
    "n_charged_atoms": ColumnAnnotation(
        description="Number of charged atoms in a molecule, computed with <datamol.descriptor.n_charged_atoms>"
    ),
    "refractivity": ColumnAnnotation(
        description="The total polarizability of a mole of a substance and is dependent on the temperature, the index of refraction, and the pressure. Computed with <datamol.descriptor.refeactivity>"
    ),
    "n_aromatic_rings": ColumnAnnotation(
        description="Number of aromatic rings in the molecule, computed with <datamol.descriptor.n_aromatic_rings>"
    ),
}

###  Define the `Dataset` object

In [8]:
dataset_version = "v2"
dataset_name = f"molprop-250k-{dataset_version}"

In [9]:
dataset = Dataset(
    table=table[annotations.keys()],
    name=dataset_name,
    description=" Molecule properties computed for ZINC15 250K dataset. Those molecular properties are used to examinate the usefullness of any pretrained models. Especially, any model for generation purpose should not fail on these tasks.",
    annotations=annotations,
    owner=owner,
    tags=["Representation", "Molecular Properties"],
    readme=load_readme("org-Polaris/molprop/molprop-250k_readme.md"),
    license="CC-BY-4.0",
    source="https://pubs.acs.org/doi/10.1021/acs.jcim.5b00559",
    user_attributes={"year": "2015"},
    curation_reference="https://github.com/polaris-hub/polaris-recipes/org-Polaris/molprop/01_molprop_data_curation.ipynb",
)

In [10]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

'gs://polaris-public/polaris-recipes/org-polaris/molprop/datasets/molprop-250k-v2/dataset.json'

In [11]:
# upload to Polaris Hub
# dataset.upload_to_hub(owner=owner, access="private")

2024-07-10 02:13:27.539 | SUCCESS  | polaris.hub.client:upload_dataset:631 - Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/polaris/molprop-250k-v2
